## Amazon Bedrock Knowledge Bases가 제공하는 고급 청킹 전략

이 노트북에서는 Amazon Bedrock Knowledge Bases가 지원하는 다음 청킹 옵션을 시연하기 위해 3개의 Knowledge Base를 생성합니다: 
1. 고정 청킹
2. 세만틱 청킹
3. 계층형 청킹
4. Lambda 함수를 활용한 사용자 정의 청킹

청킹은 임베딩 전에 텍스트를 작은 조각으로 나누는 과정입니다. 데이터 소스를 생성한 이후에는 청킹 전략을 변경할 수 없습니다. 현재 Amazon Bedrock Knowledge Bases는 기본적으로 청킹을 사용하지 않음, 고정 크기 청킹, 기본 청킹 등 일부 내장 옵션만 지원합니다.

세만틱 청킹과 계층형 청킹 기능(기존 옵션과 함께)을 사용하면 고객은 Lambda 함수를 통해 데이터가 처리되고 청킹되는 방식을 더 세밀하게 제어할 수 있습니다.

데모에는 `Octank Financial`이라는 가상의 회사에 대한 10-K 보고서를 사용합니다. Knowledge Base를 생성한 뒤 동일한 데이터 세트에서 결과를 평가하여 검색 품질을 향상시키고, 그에 따라 Foundation Model이 생성하는 응답의 정확도를 높이는 데 집중합니다. 

## 1. 필요한 라이브러리 불러오기
먼저 필요한 패키지를 설치합니다.

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
%pip install ragas==0.1.9 --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import botocore
botocore.__version__

In [ ]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name_standard = 'standard-kb'
knowledge_base_name_hierarchical = 'hierarchical-kb'
knowledge_base_name_semantic = 'semantic-kb'
knowledge_base_name_custom = 'custom-chunking-kb'
knowledge_base_description = "Knowledge Base containing complex PDF."
bucket_name = f'{knowledge_base_name_standard}-{suffix}'
intermediate_bucket_name = f'{knowledge_base_name_standard}-intermediate-{suffix}'
lambda_function_name = f'{knowledge_base_name_custom}-lambda-{suffix}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2 - 고정 청킹 전략으로 Knowledge Base 생성
먼저 레스토랑 메뉴를 저장할 [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/)를 생성해 보겠습니다. Knowledge Bases는 [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/), [Pinecone](http://app.pinecone.io/bedrock-integration), [Redis Enterprise]() 및 [MongoDB Atlas]() 등 다양한 벡터 데이터베이스와 통합할 수 있습니다. 이 예제에서는 Knowledge Base를 Amazon OpenSearch Serverless와 연동합니다. 이를 위해 Knowledge Base와 필요한 사전 준비 작업을 모두 생성해 주는 헬퍼 클래스 `BedrockKnowledgeBase`를 사용합니다:
1. IAM 역할 및 정책
2. S3 버킷
3. Amazon OpenSearch Serverless 암호화, 네트워크 및 데이터 접근 정책
4. Amazon OpenSearch Serverless 컬렉션
5. Amazon OpenSearch Serverless 벡터 인덱스
6. Knowledge Base
7. Knowledge Base 데이터 소스

먼저 고정 청킹 전략으로 Knowledge Base를 생성한 다음 계층형 청킹 전략을 적용합니다. 

매개변수 값: 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [ ]:
knowledge_base_standard = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_standard}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

## 2.1 데이터 세트를 Amazon S3에 업로드
Knowledge Base를 만들었으니 `Octank financial 10K` 보고서 데이터 세트로 내용을 채워 보겠습니다. Knowledge Base 데이터 소스는 연결된 S3 버킷에 데이터가 있어야 하며, 데이터 변경 사항은 `StartIngestionJob` API 호출을 통해 Knowledge Base와 동기화할 수 있습니다. 이 예제에서는 헬퍼 클래스를 통해 API의 [boto3 추상화](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/start_ingestion_job.html)를 사용합니다. 

먼저 `dataset` 폴더에 있는 데이터를 S3에 업로드합니다.

In [ ]:
import os

def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md"]:
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)
            else:
                print(f"Skipping file {file_to_upload}")

upload_directory("../synthetic_dataset", bucket_name)


이제 ingestion 작업을 시작합니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_standard.start_ingestion_job()

마지막으로 나중에 솔루션을 테스트할 수 있도록 Knowledge Base ID를 저장합니다.

In [ ]:
kb_id_standard = knowledge_base_standard.get_knowledge_base_id()

### 2.2 Knowledge Base 테스트
Knowledge Base가 준비되었으므로 [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용해 확인할 수 있습니다. 

#### Retrieve and Generate API로 Knowledge Base 테스트

먼저 retrieve and generate API로 Knowledge Base를 시험해 보겠습니다. 이 API는 Bedrock이 Knowledge Base에서 필요한 참조를 검색하고 Bedrock의 Foundation Model로 최종 답변을 생성합니다.

query = `Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

해당 질의의 정답(ground truth QA 쌍 기준)은 다음과 같습니다: 

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment (
```

In [ ]:
query = "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019."

In [ ]:
time.sleep(20)
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_standard,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

이처럼 Retrieve and Generate API를 사용하면 최종 응답을 바로 받을 수 있습니다. 이제 `RetrieveAndGenerate` API에서 제공하는 인용 정보를 살펴보고, 응답을 생성할 때 모델이 반환하는 검색된 청크와 인용을 확인해 보겠습니다. 질의와 함께 적절한 컨텍스트를 Foundation Model에 제공하면 고품질 응답이 생성될 가능성이 크게 높아집니다.

In [ ]:
def citations_rag_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret,1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

In [ ]:
response_standard = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_standard))
citations_rag_print(response_standard)

Knowledge Base에서 제공하는 소스 정보를 Retrieve API로 확인해 보겠습니다.

#### Retrieve API로 Knowledge Base 테스트
추가적인 제어가 필요하다면 Retrieve API를 사용해 질의와 가장 잘 일치하는 청크를 직접 가져올 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수, 청크 메타데이터를 제공합니다.

In [ ]:
def response_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)


In [ ]:
response_standard_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_standard, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)

print("# of retrieved results: ", len(response_standard_ret['retrievalResults']))
response_print(response_standard_ret)

고정 청킹을 사용할 경우 기본값인 `semantic similarity`로 5개의 검색 결과를 요청하면, API가 5개의 결과를 반환하는 것을 확인할 수 있습니다. 이제 `hierarchical chunking` 전략을 적용한 뒤 `RetrieveAndGenerate` API와 `Retrieve` API로 검색 결과를 비교해 보겠습니다.

## 3. 계층형 청킹 전략으로 Knowledge Base 생성

**개념**

계층형 청킹은 데이터를 계층 구조로 구성하여 데이터에 존재하는 관계를 기반으로 더 세밀하고 효율적인 검색이 가능하도록 합니다. 데이터가 계층 구조로 구성되면 RAG 워크플로가 복잡하고 중첩된 데이터 세트에서도 정보를 효율적으로 탐색하고 가져올 수 있습니다.
문서가 파싱된 후 첫 단계로 부모 청크와 자식 청크 크기에 따라 문서를 분할합니다. 이후 청크는 계층 구조로 정리되며, 부모 청크(상위 레벨)는 더 큰 단위(예: 문서나 섹션)를, 자식 청크(하위 레벨)는 더 작은 단위(예: 문단이나 문장)를 나타냅니다. 부모와 자식 청크 간의 관계는 유지되며, 이 구조 덕분에 코퍼스를 효율적으로 탐색하고 검색할 수 있습니다.

**이점**

- 효율적인 검색: 계층 구조는 먼저 자식 청크에 대해 세만틱 검색을 수행한 뒤 검색 시 부모 청크를 반환하므로 관련 정보를 빠르고 정확하게 찾아낼 수 있습니다. 자식 청크를 부모 청크로 교체해 제공함으로써 Foundation Model에 더 크고 포괄적인 컨텍스트를 전달합니다.
- 컨텍스트 보존: 코퍼스를 계층적으로 구성하면 청크 간의 컨텍스트 관계가 유지되어, 일관성 있고 컨텍스트에 맞는 텍스트 생성에 도움이 됩니다.

><br>          
>참고:
>계층형 청킹에서는 부모 청크가 반환되고 검색은 자식 청크에서 수행되므로, 요청한 결과 수보다 적은 검색 결과가 반환될 수 있습니다. 하나의 부모 청크에는 여러 자식 청크가 포함될 수 있기 때문입니다.
><br></br>       

계층형 청킹은 기술 매뉴얼, 법률 문서, 복잡한 형식과 중첩된 테이블을 포함한 학술 문서처럼 계층적 구조가 있는 복잡한 문서에 특히 적합합니다.

**매개변수 값:** 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [ ]:
knowledge_base_hierarchical = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_hierarchical}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    chunking_strategy = "HIERARCHICAL", 
    suffix = f'{suffix}-h'
)

이제 ingestion 작업을 시작합니다. 이미 고정 청킹에서 사용한 것과 동일한 문서를 사용하므로, S3 버킷에 문서를 업로드하는 단계는 건너뜁니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_hierarchical.start_ingestion_job()

추가 테스트를 위해 Knowledge Base ID를 저장합니다.

In [ ]:
kb_id_hierarchical = knowledge_base_hierarchical.get_knowledge_base_id()

### 3.1 Knowledge Base 테스트
Knowledge Base가 준비되었으므로 [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용해 확인할 수 있습니다. 

#### Retrieve and Generate API로 Knowledge Base 테스트

먼저 retrieve and generate API로 Knowledge Base를 시험해 보겠습니다. 이 API는 Bedrock이 Knowledge Base에서 필요한 참조를 검색하고 Bedrock의 Foundation Model로 최종 답변을 생성합니다.

query = `Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

해당 질의의 정답(ground truth QA 쌍 기준)은 다음과 같습니다: 

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
```

In [ ]:
time.sleep(20)
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_hierarchical,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

이처럼 Retrieve and Generate API를 사용하면 최종 응답을 바로 받을 수 있습니다. 이제 `RetrieveAndGenerate` API에서 제공하는 인용 정보를 살펴보고, 응답을 생성할 때 모델이 반환하는 검색된 청크와 인용을 확인해 보겠습니다. 질의와 함께 적절한 컨텍스트를 Foundation Model에 제공하면 고품질 응답이 생성될 가능성이 크게 높아집니다.

In [ ]:
response_hierarchical = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_hierarchical))
citations_rag_print(response_hierarchical)

Knowledge Base에서 제공하는 소스 정보를 Retrieve API로 확인해 보겠습니다.

#### Retrieve API로 Knowledge Base 테스트
추가적인 제어가 필요하다면 Retrieve API를 사용해 질의와 가장 잘 일치하는 청크를 직접 가져올 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수, 청크 메타데이터를 제공합니다.

In [ ]:
response_hierarchical_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_hierarchical, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)

print("# of retrieved results: ", len(response_hierarchical_ret['retrievalResults']))
response_print(response_hierarchical_ret)

><br>
> 참고:
> 위 응답에서 볼 수 있듯이 `retrieve` API는 요청에서 5개의 결과를 전달했지만 3개의 검색 결과(청크)만 반환했습니다. `hierarchical` 청킹에서는 검색이 자식 청크에서 수행되고 API는 부모 청크를 반환하므로, 하나의 부모 청크에 여러 자식 청크가 포함될 수 있습니다. 따라서 검색은 5개의 자식 청크에 대해 수행되었으나 응답에는 3개의 부모 청크만 포함되었습니다.
><br></br>

## 4. 세만틱 청킹 전략으로 Knowledge Base 생성

**개념**

세만틱 청킹은 텍스트 내부의 관계를 분석해 임베딩 모델이 계산한 의미적 유사성에 기반하여 의미 있고 완전한 청크로 분할합니다. 이 접근 방식은 검색 과정에서 정보의 온전성을 유지해 정확하고 문맥에 맞는 결과를 제공합니다.
Amazon Bedrock Knowledge Bases는 먼저 지정된 토큰 크기에 따라 문서를 청크로 분할합니다. 각 청크에 대해 임베딩을 생성하고, 임베딩 공간에서 유사성 임계값과 버퍼 크기를 기준으로 유사한 청크를 결합해 새로운 청크를 형성합니다. 그 결과 청크 크기는 청크마다 달라질 수 있습니다.

**이점**

- 텍스트의 의미와 컨텍스트에 집중함으로써 세만틱 청킹은 검색 품질을 크게 향상시킵니다. 텍스트의 의미적 일관성을 유지하는 것이 중요한 상황에서 사용하기 좋습니다.

- 이 방법은 고정 크기 청킹보다 계산 비용이 더 들지만, 컨텍스트 경계가 명확하지 않은 문서(예: 법률 문서, 기술 매뉴얼)를 청킹하는 데 유용합니다.[[1]](#https://www.mongodb.com/developer/products/atlas/choosing-chunking-strategy-rag/)

**매개변수 값:**
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [ ]:
knowledge_base_semantic = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_semantic}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source, 
    chunking_strategy = "SEMANTIC", 
    suffix = f'{suffix}-s'
)

이제 ingestion 작업을 시작합니다. 이미 고정 청킹에서 사용한 것과 동일한 문서를 사용하므로, S3 버킷에 문서를 업로드하는 단계는 건너뜁니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_semantic.start_ingestion_job()

In [ ]:
kb_id_semantic = knowledge_base_semantic.get_knowledge_base_id()

### 4.1 Knowledge Base 테스트
Knowledge Base가 준비되었으므로 [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용해 확인할 수 있습니다. 

#### Retrieve and Generate API로 Knowledge Base 테스트

먼저 retrieve and generate API로 Knowledge Base를 시험해 보겠습니다. 이 API는 Bedrock이 Knowledge Base에서 필요한 참조를 검색하고 Bedrock의 Foundation Model로 최종 답변을 생성합니다.

query = `Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

해당 질의의 정답(ground truth QA 쌍 기준)은 다음과 같습니다: 

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow. 
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.
```

In [ ]:
time.sleep(20)

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_semantic,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

이처럼 Retrieve and Generate API를 사용하면 최종 응답을 바로 받을 수 있습니다. 이제 `RetrieveAndGenerate` API에서 제공하는 인용 정보를 살펴보고, 응답을 생성할 때 모델이 반환하는 검색된 청크와 인용을 확인해 보겠습니다. 질의와 함께 적절한 컨텍스트를 Foundation Model에 제공하면 고품질 응답이 생성될 가능성이 크게 높아집니다.

In [ ]:
response_semantic = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_semantic))
citations_rag_print(response_semantic)

Knowledge Base에서 제공하는 소스 정보를 Retrieve API로 확인해 보겠습니다.

#### Retrieve API로 Knowledge Base 테스트
추가적인 제어가 필요하다면 Retrieve API를 사용해 질의와 가장 잘 일치하는 청크를 직접 가져올 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수, 청크 메타데이터를 제공합니다.

In [ ]:
response_semantic_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_semantic, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)
print("# of citations or chunks used to generate the response: ", len(response_semantic_ret['retrievalResults']))
response_print(response_semantic_ret)

## 5. Lambda 함수를 사용하는 사용자 정의 청킹 옵션
Amazon Bedrock용 Knowledge Base(KB)를 생성할 때 Lambda 함수를 연결해 사용자 지정 청킹 로직을 정의할 수 있습니다. Ingestion 동안 Lambda 함수가 제공되면 Knowledge Bases는 해당 함수를 실행하고 입력 및 출력 값을 제공한 중간 S3 버킷에 저장합니다.

> <br>
> 참고: Lambda 함수는 사용자 정의 청킹 로직을 추가하는 용도뿐만 아니라 청크 수준 메타데이터를 추가하는 등 청크 후처리를 수행하는 데에도 사용할 수 있습니다. 이 예제에서는 Lambda 함수를 사용자 정의 청킹 로직에 초점을 맞춰 사용합니다.
> <br></br>

### 5.1 Lambda 함수 생성

이제 사용자 정의 청킹 로직이 포함된 Lambda 함수를 생성하겠습니다. 이를 위해 다음 작업을 수행합니다:

1. 사용자 정의 청킹 로직을 포함하는 `lambda_function.py` 파일을 생성합니다.
2. Lambda 함수용 IAM 역할을 생성합니다.
3. 필요한 권한과 함께 Lambda 함수를 생성합니다.

#### 함수 코드 생성
Lambda 함수가 중간 버킷에서 파일을 읽고, 사용자 정의 청킹 로직으로 내용을 처리한 뒤 결과를 다시 S3 버킷에 쓰도록 구현합니다.

In [ ]:
%%writefile lambda_function.py
import json
from abc import abstractmethod, ABC
from typing import List
from urllib.parse import urlparse
import boto3
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

class Chunker(ABC):
    @abstractmethod
    def chunk(self, text: str) -> List[str]:
        raise NotImplementedError()
        
class SimpleChunker(Chunker):
    def chunk(self, text: str) -> List[str]:
        words = text.split()
        return [' '.join(words[i:i+100]) for i in range(0, len(words), 100)]

def lambda_handler(event, context):
    logger.debug('input={}'.format(json.dumps(event)))
    s3 = boto3.client('s3')

    # Extract relevant information from the input event
    input_files = event.get('inputFiles')
    input_bucket =  event.get('bucketName')

    
    if not all([input_files, input_bucket]):
        raise ValueError("Missing required input parameters")
    
    output_files = []
    chunker = SimpleChunker()

    for input_file in input_files:
        content_batches = input_file.get('contentBatches', [])
        file_metadata = input_file.get('fileMetadata', {})
        original_file_location = input_file.get('originalFileLocation', {})

        processed_batches = []
        
        for batch in content_batches:
            input_key = batch.get('key')

            if not input_key:
                raise ValueError("Missing uri in content batch")
            
            # Read file from S3
            file_content = read_s3_file(s3, input_bucket, input_key)
            
            # Process content (chunking)
            chunked_content = process_content(file_content, chunker)
            
            output_key = f"Output/{input_key}"
            
            # Write processed content back to S3
            write_to_s3(s3, input_bucket, output_key, chunked_content)
            
            # Add processed batch information
            processed_batches.append({
                'key': output_key
            })
        
        # Prepare output file information
        output_file = {
            'originalFileLocation': original_file_location,
            'fileMetadata': file_metadata,
            'contentBatches': processed_batches
        }
        output_files.append(output_file)
    
    result = {'outputFiles': output_files}
    
    return result
    

def read_s3_file(s3_client, bucket, key):
    response = s3_client.get_object(Bucket=bucket, Key=key)
    return json.loads(response['Body'].read().decode('utf-8'))

def write_to_s3(s3_client, bucket, key, content):
    s3_client.put_object(Bucket=bucket, Key=key, Body=json.dumps(content))    

def process_content(file_content: dict, chunker: Chunker) -> dict:
    chunked_content = {
        'fileContents': []
    }
    
    for content in file_content.get('fileContents', []):
        content_body = content.get('contentBody', '')
        content_type = content.get('contentType', '')
        content_metadata = content.get('contentMetadata', {})
        
        words = content['contentBody']
        chunks = chunker.chunk(words)
        
        for chunk in chunks:
            chunked_content['fileContents'].append({
                'contentType': content_type,
                'contentMetadata': content_metadata,
                'contentBody': chunk
            })
    
    return chunked_content

Knowledge Bases에서 제공하는 표준 청킹 전략 값은 다음과 같습니다: 

**매개변수 값:**
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

사용자 정의 로직을 구현하기 위해 `knowledge_base.py` 클래스에 `CUSTOM` 값을 전달할 수 있는 옵션을 추가했습니다. 
이 클래스에서 청킹 전략을 `CUSTOM`으로 전달하면 다음 작업이 수행됩니다: 

1. `chunkingStrategy`를 `NONE`으로 설정합니다. 
2. `vectorIngestionConfiguration`에 `customTransformationConfiguration`을 다음과 같이 추가합니다: 

```
{
...
   "vectorIngestionConfiguration": {
    "customTransformationConfiguration": { 
         "intermediateStorage": { 
            "s3Location": { 
               "uri": "string"
            }
         },
         "transformations": [
            {
               "transformationFunction": {
                  "lambdaConfiguration": {
                     "lambdaArn": "string"
                  }
               },
               "stepToApply": "string" // enum of POST_CHUNKING
            }
         ]
      },
      "chunkingConfiguration": {
         "chunkingStrategy": "NONE"
         ...
   }
}

```

In [ ]:
knowledge_base_custom = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_custom}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    lambda_function_name=lambda_function_name,
    intermediate_bucket_name=intermediate_bucket_name, 
    chunking_strategy = "CUSTOM", 
    suffix = f'{suffix}-c'
)

이제 ingestion 작업을 시작합니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_custom.start_ingestion_job()

In [ ]:
kb_id_custom = knowledge_base_custom.get_knowledge_base_id()

### 5.2 Knowledge Base 테스트
Knowledge Base가 준비되었으므로 [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용해 확인할 수 있습니다. 

#### Retrieve and Generate API로 Knowledge Base 테스트

먼저 retrieve and generate API로 Knowledge Base를 시험해 보겠습니다. 이 API는 Bedrock이 Knowledge Base에서 필요한 참조를 검색하고 Bedrock의 Foundation Model로 최종 답변을 생성합니다.

query = `Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

해당 질의의 정답(ground truth QA 쌍 기준)은 다음과 같습니다: 

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
```

In [ ]:
time.sleep(10)

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_custom,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

이처럼 Retrieve and Generate API를 사용하면 최종 응답을 바로 받을 수 있습니다. 이제 `RetrieveAndGenerate` API에서 제공하는 인용 정보를 살펴보고, 응답을 생성할 때 모델이 반환하는 검색된 청크와 인용을 확인해 보겠습니다. 질의와 함께 적절한 컨텍스트를 Foundation Model에 제공하면 고품질 응답이 생성될 가능성이 크게 높아집니다.

In [ ]:
response_custom = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_custom))
citations_rag_print(response_custom)

Knowledge Base에서 제공하는 소스 정보를 Retrieve API로 확인해 보겠습니다.

#### Retrieve API로 Knowledge Base 테스트
추가적인 제어가 필요하다면 Retrieve API를 사용해 질의와 가장 잘 일치하는 청크를 직접 가져올 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수, 청크 메타데이터를 제공합니다.

In [ ]:
response_custom_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_custom, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)
print("# of citations or chunks used to generate the response: ", len(response_custom_ret['retrievalResults']))
response_print(response_custom_ret)

모든 경우에서 단일 질의를 평가했을 때 올바른 응답을 얻었습니다. 그러나 RAG 애플리케이션을 구축할 때는 정확도 향상을 확인하기 위해 다수의 질문과 답변으로 평가해야 합니다. 다음 단계에서는 오픈소스 프레임워크인 RAG Assessment(RAGAS)를 사용해 `your dataset`에 대한 응답을 평가하고, 컨텍스트 품질 또는 검색 결과 품질과 관련된 지표를 살펴봅니다.
우리는 다음 두 가지 지표에 집중합니다: 

1. Context recall
2. Context relevancy

## 6. RAG Assessment(RAGAS) 프레임워크로 데이터 세트의 검색 결과 평가
각 청킹 전략에 대해 RAGAS 프레임워크를 사용해 결과를 평가할 수 있습니다. 이 접근 방식은 데이터 세트에 어떤 청킹 전략을 적용해야 하는지 객관적인 근거를 제공해 줍니다. 

이상적으로는 다른 매개변수도 함께 최적화해야 합니다. 예를 들어 계층형 청킹의 경우 부모 청크와 자식 청크 크기를 다양하게 시도해 보세요. 

아래 접근 방식은 Amazon Bedrock Knowledge Bases가 권장하는 기본 매개변수에 기반해 어떤 전략을 사용할지에 대한 휴리스틱을 제공합니다. 

In [ ]:
print("Semantic: ", kb_id_semantic)
print("Standard: ", kb_id_standard)
print("Hierarchical: ", kb_id_hierarchical)
print("Custom chunking: ", kb_id_custom)

#### 평가
이 섹션에서는 RAGAS를 사용해 다음 지표로 검색 결과를 평가합니다:
1. **Context Recall:** 검색된 컨텍스트가 그라운드 트루스로 간주되는 주석 답변과 얼마나 일치하는지를 측정합니다. 그라운드 트루스와 검색된 컨텍스트를 기반으로 계산하며 0에서 1 사이의 값을 가지며, 값이 높을수록 성능이 좋습니다.

2. **Context relevancy:** 이 지표는 질문과 컨텍스트를 바탕으로 검색된 컨텍스트의 관련성을 측정합니다. 값은 (0, 1) 범위에 있으며, 값이 높을수록 관련성이 높습니다.

In [ ]:
from utils.evaluation import KnowledgeBasesEvaluations

from ragas.metrics import (
    context_recall,
    context_precision,
    )

metrics = [context_recall,
           context_precision
           ]

MODEL_ID_EVAL = "anthropic.claude-3-sonnet-20240229-v1:0"
MODEL_ID_GEN = "anthropic.claude-3-haiku-20240307-v1:0"

questions = [
        "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.",
]
ground_truths = [
    "The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:\
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.\
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).\
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow. \
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.",
]
kb_evaluate_standard = KnowledgeBasesEvaluations(model_id_eval=MODEL_ID_EVAL, 
                        model_id_generation=MODEL_ID_GEN, 
                        metrics=metrics,
                        questions=questions, 
                        ground_truth=ground_truths, 
                        KB_ID=kb_id_standard,
                        )

kb_evaluate_hierarchical = KnowledgeBasesEvaluations(model_id_eval=MODEL_ID_EVAL, 
                        model_id_generation=MODEL_ID_GEN, 
                        metrics=metrics,
                        questions=questions, 
                        ground_truth=ground_truths, KB_ID=kb_id_hierarchical)

kb_evaluate_semantic = KnowledgeBasesEvaluations(model_id_eval=MODEL_ID_EVAL, 
                        model_id_generation=MODEL_ID_GEN, 
                        metrics=metrics,
                        questions=questions, 
                        ground_truth=ground_truths, KB_ID=kb_id_semantic)

kb_evaluate_custom = KnowledgeBasesEvaluations(model_id_eval=MODEL_ID_EVAL, 
                        model_id_generation=MODEL_ID_GEN, 
                        metrics=metrics,
                        questions=questions, 
                        ground_truth=ground_truths, KB_ID=kb_id_custom)

In [ ]:
results_heirarchical = kb_evaluate_hierarchical.evaluate()
results_standard = kb_evaluate_standard.evaluate()
results_semantic = kb_evaluate_semantic.evaluate()
results_custoom = kb_evaluate_custom.evaluate()

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 800
print("Fixed Chunking Evaluation for synthetic 10K report")
print("--------------------------------------------------------------------")
print("Average context_recall: ", results_standard["context_recall"].mean())
print("Average context_relevancy: ", results_standard["context_precision"].mean(), "\n")

print("Hierarchical Chunking Evaluation for synthetic 10K report")
print("--------------------------------------------------------------------")
print("Average context_recall: ", results_heirarchical["context_recall"].mean())
print("Average context_relevancy: ", results_heirarchical["context_precision"].mean(), "\n")

print("Semantic Chunking Evaluation for synthetic 10K report")
print("--------------------------------------------------------------------")
print("Average context_recall: ", results_semantic["context_recall"].mean())
print("Average context_relevancy: ", results_semantic["context_precision"].mean(), "\n")

print("Custom Chunking Evaluation for synthetic 10K report")
print("--------------------------------------------------------------------")
print("Average context_recall: ", results_custoom["context_recall"].mean())
print("Average context_relevancy: ", results_custoom["context_precision"].mean())


In [ ]:
print("===============================Knowledge base with fixed chunking==============================\n")
knowledge_base_standard.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)
print("===============================Knowledge base with hierarchical chunking==============================\n")
knowledge_base_hierarchical.delete_kb(delete_s3_bucket=False,delete_iam_roles_and_policies=True)
print("===============================Knowledge base with semantic chunking==============================\n")
knowledge_base_semantic.delete_kb(delete_s3_bucket=False,delete_iam_roles_and_policies=True)
print("===============================Knowledge base with custom chunking==============================\n")
knowledge_base_custom.delete_kb(delete_s3_bucket=True,delete_iam_roles_and_policies=True, delete_lambda_function = True)